Dependencies

In [10]:
%pip install pandas
%pip install fastparquet
%pip install tensorflow
%pip install matplotlib
%pip install numpy
%pip install keras

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Setup

In [1]:
import re
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

Import data

In [5]:
df = pd.read_parquet("data/Thermo_SRM_Pool_meta_data.parquet", engine='fastparquet')

In [7]:
print(df.shape)
print(df.head(4))
print(df.columns)

(6080606, 15)
                                       raw_file  scan_number  \
0  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        34736   
1  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        25591   
2  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        24831   
3  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        34870   

      modified_sequence  precursor_charge  precursor_intensity          mz  \
0  LPGSLETYVEQEQGENANDR                 2           29525630.0  1125.01950   
1       HGSLQEYLQNDTGSK                 2           13188580.0   838.89737   
2        VEEEEEINSELTAR                 2           20663460.0   824.38904   
3  LPGSLETYVEQEQGENANDR                 2           19884630.0  1125.01950   

   precursor_mz fragmentation mass_analyzer  retention_time  \
0   1125.018331           CID          ITMS          38.411   
1    838.894401           HCD          FTMS          30.394   
2    824.388401           CID          ITMS          29.737   
3   1125.018831       

Check what charge states are in the data

In [8]:
list_charge = df["precursor_charge"].unique().tolist()
print(sorted(list_charge))

[1, 2, 3, 4, 5, 6, 7]


Create dataframe containing only sequence and precursor charge

In [9]:
# drop all columns we dont need for training
for column in df.columns:
    if column not in ["modified_sequence", "precursor_charge"]:
        df.drop(column, axis=1, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6080606 entries, 0 to 6080605
Data columns (total 2 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   modified_sequence  object
 1   precursor_charge   int64 
dtypes: int64(1), object(1)
memory usage: 92.8+ MB


Split data into two classes:
-   class with only precursor charge 2
-   class with every other precursor charge

calculate weights to correct for class imbalance

In [2]:
# CHANGE TO REAL COUNTS LATER!! JUST TO GET RID OF ANNOYING SQUIGGLY LINES IN THE BEGINNING
COUNT_DoubleCharge = 4
COUNT_OtherCharges = 12

In [3]:
initial_bias = np.log([COUNT_DoubleCharge / COUNT_OtherCharges])
print("Initial bias: {:.5f}".format(initial_bias[0]))

TRAINING_COUNT = COUNT_DoubleCharge + COUNT_OtherCharges
weight_for_other = (1 / COUNT_OtherCharges) * TRAINING_COUNT / 2.0
weight_for_double = (1 / COUNT_DoubleCharge) * TRAINING_COUNT / 2.0

print("weight for Double Charge: {:.2f}".format(weight_for_double))
print("weight for Other Charges: {:.2f}".format(weight_for_other))

Initial bias: -1.09861
weight for Double Charge: 2.00
weight for Other Charges: 0.67


NameError: name 'test' is not defined